In [4]:
import pandas as pd 

sample_definition = "all_samples_per_participant"
dimensions_selection = "all_dimensions"
dimension = "all"
score = "r2"

data_scores = pd.read_feather(f"../../all_data/age_prediction_performances/scores_{sample_definition}_{dimensions_selection}.feather").to_dict()

In [5]:
pd.DataFrame(data_scores)

,dimension,subdimension,sub_subdimension,algorithm,sample_size,rmse,rmse_std,r2,r2_std
0,*,*,*,*,83629,4.592113,0.012090,0.660751,0.001829
1,*instances01,*,*,*,4404,3.015260,0.035237,0.869851,0.003602
2,*instances1.5x,*,*,*,77269,4.709071,0.012408,0.635053,0.001931
3,*instances23,*,*,*,1956,2.263247,0.041027,0.900849,0.003993
4,Abdomen,*,*,*,36700,3.700446,0.014485,0.763341,0.002007
...,...,...,...,...,...,...,...,...,...
326,PhysicalActivity,FullWeek,TimeSeriesFeatures,*,98222,6.112644,0.014212,0.417744,0.002605
327,PhysicalActivity,FullWeek,TimeSeriesFeatures,1DCNN,98222,6.112644,0.014489,0.417744,0.002625
328,PhysicalActivity,Walking,*,*,98215,6.224478,0.012967,0.396758,0.002419
329,PhysicalActivity,Walking,3D,*,98215,6.594051,0.012083,0.322997,0.001549
